# Control Minitaur with Sliders live and plot actions live

## imports

In [7]:
#import pybullet_envs
import os
bullet_dir = r"C:\coding_repos\bullet3\examples\pybullet\gym"
print("parentdir=", bullet_dir)
os.sys.path.insert(0, bullet_dir)
import pybullet as pb
import math

from pybullet_envs.minitaur.envs.minitaur_gym_env import MinitaurGymEnv
import pandas as pd, numpy as np
import time
import matplotlib.pyplot as plt
from jupyter_ui_poll import ui_events


from ipywidgets import interact, interactive, fixed, interact_manual, Layout, VBox, HBox
import ipywidgets as widgets


%load_ext autoreload
%autoreload 2
%xmode Minimal

parentdir= C:\coding_repos\bullet3\examples\pybullet\gym
Exception reporting mode: Minimal


# Minitaur: manual steering in leg space

In [2]:
env = MinitaurGymEnv(
      urdf_version=minitaur_gym_env.DERPY_V0_URDF_VERSION,
      render=True,
      motor_velocity_limit=np.inf,
      pd_control_enabled=True,
      hard_reset=True,
      on_rack=False,
      )

C:\Users\kaiae\.conda\envs\doggo\lib\site-packages\gym\logger.py:30: UserWarning:

WARN: Box bound precision lowered by casting to float32



In [3]:
%matplotlib widget

In [4]:
motor_names = [
 'motor_front_leftL_joint',
 'motor_front_leftR_joint',
 'motor_back_leftL_joint',
 'motor_back_leftR_joint',
 'motor_front_rightL_joint',
 'motor_front_rightR_joint',
 'motor_back_rightL_joint',
 'motor_back_rightR_joint']

In [5]:
leg_model_input_description = [
 's_front_left',
 's_back_left',
 's_front_right',
 's_back_right',
 'e_front_left',
 'e_back_left',
 'e_front_right',
 'e_back_right']

In [8]:
action = [0]*8
plt.close('all')

plt.ioff()

    
output = widgets.Output(layout={'width': '700px', 'height': '300px'})
d_desc_to_pos = {d:i for i, d in enumerate(leg_model_input_description)}

# initialise leg space to 0
leg_angles = [0] * 8



fig, axs= plt.subplots(8, 2, figsize=(10, 8), sharex=True)

fig.canvas.header_visible = False
fig.canvas.toolbar_visible = False

lines = []
for i in range(8):    
    axs[i,0].set_ylim(-1.5,1.5)
    #axs[i,0].set_xlim(0,55)
    axs[i,1].set_ylim(-180,180)
    #axs[i,1].set_xlim(0,55)
    
# index giver
from itertools import count
x_value = count()

# expanding dataset
x, y_desired, y_actual = [], [], []

# initialise dummy data
[x.append(next(x_value)) for i in range(1)]
[y_desired.append(action) for i in range(1)]
[y_actual.append(action) for i in range(1)]

# setup desired and actual angle plots
dfs=[]
ax_df_desired = pd.DataFrame(index=x,columns=leg_model_input_description, 
             data=y_desired).plot(subplots=True, ax=axs[:,0])

ax_df_actual = pd.DataFrame(index=x,columns=motor_names, 
             data=y_actual).plot(subplots=True, ax=axs[:,1])
dfs.append(ax_df_desired)
dfs.append(ax_df_actual)

# monitor framerate
t_start = time.time()        
        
# slider event handler
def on_value_changed(newval):
    with output:
        #print(newval)
        desc, val = newval.owner.description, newval.new
        pos = d_desc_to_pos[desc]
        leg_angles[pos] = val        

        global action
        action = leg_angles

        df_actual = pd.DataFrame(columns=["pos", "vel", "reaction forces", "motor torque"], 
                                data=pb.getJointStates(env.minitaur.quadruped, range(len(d_desc_to_pos))))

        
        
        next_x = next(x_value)
        x.append(next_x)
        y_desired.append(leg_angles[:]) # need to make copy else it doesnt work, to do with globals?
        y_actual.append(df_actual.loc[:,"pos"].mul(180).div(math.pi).tolist())
        
        # update graph live    
        for i in range(8):
            #la = dfs[0][i].get_lines()[0]
            dfs[0][i].get_lines()[0].set_data(x, pd.DataFrame(y_desired).iloc[:,i])
            #lb = dfs[1][i].get_lines()[0]
            dfs[1][i].get_lines()[0].set_data(x, pd.DataFrame(y_actual).iloc[:,i])

            dfs[0][i].autoscale_view(None,'x',None)
            dfs[1][i].autoscale_view(None,'x',None)
            dfs[0][i].relim()
            dfs[1][i].relim()
            
            
        fig.canvas.flush_events()
        fig.canvas.draw()
        
        print(f"FPS: {round(next_x/(time.time() - t_start),2)}", end=", ")
        
        
        




# create control sliders for each motor and attach event handlers
sliders = []
s_min, s_max = -1, 1
for slider in list(d_desc_to_pos.keys()):
    int_slider = widgets.FloatSlider(description=slider, 
                                     min=s_min, max=s_max, 
                                     value = 0, continuous_update=True,
                                     orientation="vertical",                                      
                                     layout=Layout(width="80px", height="400px"))    
    int_slider.observe(on_value_changed, names="value")
    sliders.append(int_slider)
sliders.append(output)



# link front left and right extension
widgets.jslink((sliders[0], 'value'), (sliders[2], 'value'))
widgets.jslink((sliders[1], 'value'), (sliders[3], 'value')) # link back left and right 
widgets.jslink((sliders[4], 'value'), (sliders[6], 'value')) # link back left and right 
widgets.jslink((sliders[5], 'value'), (sliders[7], 'value')) # link back left and right 

# layout and display sliders
box_layout = Layout(display='flex', flex_flow='row', justify_content='space-between', align_items='stretch', width="100%")
sliders = widgets.HBox([widgets.HBox(sliders), fig.canvas], layout=box_layout)
display(sliders)
fig.subplots_adjust(left=0.03, top=1, hspace=0) # larger top values decreases top space



env.reset()
pb.setRealTimeSimulation(0)
obs_history = []
with ui_events() as poll:
    while action is not None:
        poll(10) # poll for 10 ui events
        #print(action, end="")
        obs, rew, done, info = env.step(action)
        obs_history.append(obs)
        #print('x', end='')
        #time.sleep(0.1)
print('done')

KeyboardInterrupt: 